In [ ]:
import numpy as np
import re
#import numpy.linalg.norm as norm
import copy
from scipy.signal import savgol_filter,argrelextrema

# Define functions

In [ ]:
# Mass percentage for men: (Reference: 'Center of Mass of Human’s Body Segments')
# Head(+neck): 6.94, Trunk: 43.46, Upper/Mid/Lower-Trunk: 15.96/16.33/11.17
# UpperArm(single): 2.71, Forearm(single): 1.62, Hand(single): 0.61
# Thigh(single): 14.16, Shank(single): 4.33, Foot(single): 1.37
# Similar for women

def calcCoM(kp,gender=1):
  '''
  kp:17x3
  gender: 0--Female, 1--Male
  CenterOfMass
  Centers: head+neck(0),trunk(1),
       upperarm_l(2),forearm_l(3),hand_l(4),
       upperarm_r(5),forearm_r(6),hand_r(7),
       thigh_l(8),shank_l(9),foot_l(10),
       thigh_r(11),shank_r(12),foot_r(13)
  '''
  if gender==0:
    vMassWeights = [6.68,42.57,2.55,1.38,0.56,2.55,1.38,0.56,
                    14.78,4.81,1.29,14.78,4.81,1.29]
  else:
    vMassWeights = [6.94,43.46,2.71,1.62,0.61,2.71,1.62,0.61,
                    14.16,4.33,1.37,14.16,4.33,1.37]
  
  CoM = np.zeros(kp.shape[:-2]+(3,))
  
  CoM += kp[...,9,:] * vMassWeights[0] # head+neck
  CoM += kp[...,7,:] * vMassWeights[1] # trunk
  CoM += (kp[...,11,:] + kp[...,12,:])/2 * vMassWeights[2] # upperarm_l
  CoM += (kp[...,12,:] + kp[...,13,:])/2 * vMassWeights[3] # forearm_l
  CoM += kp[...,13,:] * vMassWeights[4] # hand_l
  CoM += (kp[...,14,:] + kp[...,15,:])/2 * vMassWeights[5] # upperarm_r
  CoM += (kp[...,15,:] + kp[...,16,:])/2 * vMassWeights[6] # forearm_r
  CoM += kp[...,16,:] * vMassWeights[7] # hand_r
  CoM += (kp[...,4,:] + kp[...,5,:])/2 * vMassWeights[8] # thigh_l
  CoM += (kp[...,5,:] + kp[...,6,:])/2 * vMassWeights[9] # shank_l
  CoM += kp[...,6,:] * vMassWeights[10] # foot_l
  CoM += (kp[...,1,:] + kp[...,2,:])/2 * vMassWeights[11] # thigh_l
  CoM += (kp[...,2,:] + kp[...,3,:])/2 * vMassWeights[12] # shank_l
  CoM += kp[...,3,:] * vMassWeights[13] # foot_l

  CoM = CoM/100

  return CoM

In [ ]:
def kineticTrajectory(poses_np, L_ON, R_ON):
  '''rtype: trajectory'''
  traj = copy.deepcopy(poses)
  total_frames = min((poses.shape[0],len(L_ON),len(R_ON)))
  idx_start = 0
  idx_end = 0
  for i in range(total_frames):
    if idx_start==0 and L_ON[i]==0 and R_ON[i]==0:
      continue
    elif idx_start==0:
      idx_start = i

In [ ]:
def step_Phase1(poses,CoMs,on_foot,pos_init=None):
  #T = np.zeros((poses.shape[0],3))
  poses_corrected = np.zeros(poses.shape)
  CoMs_corrected = np.zeros(CoMs.shape)
  if pos_init is None:
    pos_init = np.array([0,0,0])
  if on_foot=='R':
    idx_foot = 3
  else:
    idx_foot = 6
  for i in range(CoMs.shape[0]):
    T = pos_init - poses[i,idx_foot,:]
    poses_corrected[i,...] = poses[i,...] + T
    #print(CoM_corrected[i,...],CoM[i,...],T)
    CoMs_corrected[i,...] = CoMs[i,...] + T
  #VoM_end = CoMs_corrected[-1,...] - CoMs_corrected[-2,...]
  #VoM_start = CoMs_corrected[1,...] - CoMs_corrected[0,...]
  VoM = (CoMs_corrected[-1,...] - CoMs_corrected[0,...])/(CoMs.shape[0]-1)
  return poses_corrected,CoMs_corrected,VoM #VoM_start,VoM_end

In [ ]:
def pred_foot_on_ground(poses):
  KeyPointNames = ['Hip','RHip','RKnee','RAnkle','LHip','LKnee',
                 'LAnkle','Spine','Thorax','Nose','Head','LShoulder',
                 'LElbow','LWrist','RShoulder','RElbow','RWrist']
  idR = KeyPointNames.index('RAnkle')
  idL = KeyPointNames.index('LAnkle')

  rKP = savgol_filter(poses[:,idR,1], window_length=9, polyorder=5)
  lKP = savgol_filter(poses[:,idL,1], window_length=9, polyorder=5)

  rmax = argrelextrema(rKP,np.greater)[0].tolist()
  lmax = argrelextrema(lKP,np.greater)[0].tolist() 

  side = 5

  rmax_fake = []
  for idx in rmax:
    if rKP[idx] < lKP[idx]:
      rmax_fake.append(idx)
    elif rKP[idx] < max(rKP[max([0,idx-side]):min(len(rKP),idx+side)]):
      rmax_fake.append(idx)

  for v in rmax_fake:
    rmax.remove(v)

  lmax_fake = []
  for idx in lmax:
    if lKP[idx] < rKP[idx]:
      lmax_fake.append(idx)
    elif lKP[idx] < max(lKP[max([0,idx-side]):min(len(lKP),idx+side)]):
      lmax_fake.append(idx)

  for v in lmax_fake:
    lmax.remove(v)
  
  '''Threshold = 10
  for v in copy.copy(rmax):
    if v-1>0 and rKP[v] - rKP[v-1]<Threshold and v-1 not in rmax:
      rmax.append(v-1)
    if v-2>0 and rKP[v] - rKP[v-2]<Threshold and v-2 not in rmax:
      rmax.append(v-2)
    if v+1<len(rKP)-1 and rKP[v] - rKP[v+1]<Threshold and v+1 not in rmax:
      rmax.append(v+1)
    if v+2<len(rKP)-1 and rKP[v] - rKP[v+2]<Threshold and v+2 not in rmax:
      rmax.append(v+2)
  
  for v in copy.copy(lmax):
    if v-1>0 and lKP[v] - lKP[v-1]<Threshold and v-1 not in lmax:
      lmax.append(v-1)
    if v-2>0 and lKP[v] - lKP[v-2]<Threshold and v-2 not in lmax:
      lmax.append(v-2)
    if v+1<len(lKP)-1 and lKP[v] - lKP[v+1]<Threshold and v+1 not in lmax:
      lmax.append(v+1)
    if v+2<len(lKP)-1 and lKP[v] - lKP[v+2]<Threshold and v+2 not in lmax:
      lmax.append(v+2)'''

  tmp = []
  Threshold = 10

  for v in copy.copy(rmax):
    if v>len(rKP)-15:
      tmp.append(v)
    else:
      if v>0 and v-1 not in rmax:
        rmax.append(v-1)
      if v<len(rKP)-1 and v+1 not in rmax:
        rmax.append(v+1)
      if v-2>0 and rKP[v] - rKP[v-2]<Threshold and v-2 not in rmax:
        rmax.append(v-2)
      if v+2<len(rKP)-1 and rKP[v] - rKP[v+2]<Threshold and v+2 not in rmax:
        rmax.append(v+2)
  
  for v in copy.copy(lmax):
    if v>len(lKP)-15:
      tmp.append(v)
    else:
      if v>0 and v-1 not in lmax:
        lmax.append(v-1)
      if v<len(rKP)-1 and v+1 not in lmax:
        lmax.append(v+1)
      if v-2>0 and lKP[v] - lKP[v-2]<Threshold and v-2 not in lmax:
        lmax.append(v-2)
      if v+2<len(lKP)-1 and lKP[v] - lKP[v+2]<Threshold and v+2 not in lmax:
        lmax.append(v+2)
  
  for v in tmp:
    if v in rmax: rmax.remove(v)
    if v in lmax: lmax.remove(v)
  #print('rmax',rmax)
  #print('lmax',lmax)

  signal = []
  for i in range(len(rKP)):
    if i in rmax and i in lmax:
      signal.append('D')
    elif i in rmax:
      signal.append('R')
    elif i in lmax:
      signal.append('L')
    else:
      signal.append('N')
  
  '''idx = -1
  while signal[idx]=='N':
    signal[idx] = 'D'
    idx -= 1'''
  
  '''level = (sum(rKP[rmax])+sum(lKP[lmax]))/(len(rmax)+len(lmax))
  
  if signal[-1] == 'N':
    signal[-1] = 'D'
    idx = -2
    while signal[idx]=='N' and (rKP[idx]>level or lKP[idx]>level):
      signal[idx] = 'D'
      idx -= 1'''
  '''idx = max([max(rmax),max(lmax)])
  while idx<len(rKP):
    signal[idx] = 'D'
    idx += 1
  idx = max([max(rmax),max(lmax)])
  while signal[idx-1] != 'N':
    signal[idx-1] = 'D'
    idx -= 1'''
  
  try:
    start = min(tmp)
    for i in range(start,len(rKP)):
      signal[i] = 'D'
  except:
    signal[-1] = 'D'
  
  return signal
  

# Main body

In [ ]:
!rm -rf 3D_estimates/
!unzip 3D_estimates.zip

# Use corrected 2D to predicting foot-ground contact
!rm -rf corrected_2D/
!unzip corrected_2D.zip

In [ ]:
# Load foot ground contact signal (ground truth)
meta_info = np.load('foot_ground_contact.npz', allow_pickle=True)['data']

In [ ]:
# Defining constants and parameters
PAIRS = [[0,1],[1,2],[2,3],[0,4],[4,5],[5,6],[0,7],[7,8],[8,9],[9,10],[8,11],
         [11,12],[12,13],[8,14],[14,15],[15,16]]
LINKS = ['RPelvis','RThigh','RCalf','LPelvis','LThigh','LCalf','Spine2',
         'Spine1','Neck','Head','LClavicle','LUpperArm','LForearm','RClavicle',
         'RUpperArm','RForearm']

Genders = {'Caterine Ibarguen': 0,
           'Yulimar Rojas': 0,
           'Olga Rypakova': 0,
           'Christian Taylor': 1,
           'Will Claye': 1,
           'Dong Bin': 1}

Frame_rate = 25
dt = 1 / Frame_rate

In [ ]:
# Main body
jump = []
y = []
res1 = []
res2 = []

FootGroundGT = True

for info in meta_info:
  print('Athlete:',info['athlete'])
  print('Jump:', info['id'])
  print('Result:', info['result'])

  if info['id'] != 'OR_3_A': continue

  f = open("./3D_estimates/"+info['id']+'.txt', "r")
  data = f.readlines()
  poses = []
  for i,d in enumerate(data):
    if i%17==0:
      keypoints = []
    kp = re.sub("\s+", " ",d.strip("[ \n]")).split()
    kp = [float(i) for i in kp]
    keypoints.append(kp)
    if i%17==16:
      poses.append(keypoints)
  poses_np = np.array(poses)

  link_lengths = dict()
  for idx,link in enumerate(LINKS):
    link_lengths[link] = np.linalg.norm(poses_np[:,PAIRS[idx][0],:] - poses_np[:,PAIRS[idx][1],:],axis=1)

  #print('Bone Name\t Lenght Mean    \t Length Std       \t Std/Mean (%)')
  bone_lengths = dict()
  for link in link_lengths.keys():
    m = np.mean(link_lengths[link])
    bone_lengths[link] = m
    s = np.std(link_lengths[link])
    #print(link, ': ', '\t', m, '\t', s, '\t', s/m*100, '%')
  #print(bone_lengths)

  H = (bone_lengths['RThigh']+bone_lengths['RCalf']+bone_lengths['LThigh']+bone_lengths['LCalf'])/2+bone_lengths['Spine1']+bone_lengths['Spine2']+bone_lengths['Neck']+bone_lengths['Head']
  ratio = info['height']/H

  Total_lengths = []
  for i in range(poses_np.shape[0]):
    total_length = 0
    for link in LINKS:
      total_length += link_lengths[link][i]
    #print(i,':',total_length)
    Total_lengths.append(total_length)
  #print(np.mean(Total_lengths),np.std(Total_lengths))

  for i in range(poses_np.shape[0]):
    poses_np[i,...] = poses_np[i,...]/Total_lengths[i]*np.mean(Total_lengths)*ratio
  
  CoMs = calcCoM(poses_np, gender=Genders[info['athlete']])

  if FootGroundGT:
    signal = info['flags']
  else:
    pose2D = np.load('corrected_2D/'+info['id']+'.npz', allow_pickle=True)['reconstruction']
    signal = pred_foot_on_ground(pose2D)
    print(signal)

  L_ON = []
  R_ON = []
  for s in signal:
    if s=='L' or s=='D': L_ON.append(1)
    else: L_ON.append(0)

    if s=='R' or s=='D': R_ON.append(1)
    else: R_ON.append(0)

  on_foot = [signal[0]]
  prev = 0
  next = 0
  periods = []
  for i in range(1,len(signal)):
    if signal[i] != signal[i-1]:
      on_foot.append(signal[i])
      next = i-1
      periods.append([prev,next])
      prev = i
  next = len(signal) - 1
  periods.append([prev,next])

  phases = []
  for s in on_foot:
    if s=='N': phases.append(1)
    else: phases.append(0)

  #print('Periods:',periods)
  #print(on_foot)
  #print(phases)


  if phases[0] == 1:
    on_foot.pop(0)
    phases.pop(0)
    periods.pop(0)
    start = periods[0][0]
    L_ON = L_ON[start:]
    R_ON = R_ON[start:]
    poses_np = poses_np[start:,...]

  if phases[0]==0 and periods[0][0]==periods[0][1]:
    on_foot.pop(0)
    on_foot.pop(0)
    phases.pop(0)
    phases.pop(0)
    periods.pop(0)
    periods.pop(0)
    start = periods[0][0]
    L_ON = L_ON[start:]
    R_ON = R_ON[start:]
    poses_np = poses_np[start:,...]


  #print(periods)
  #print(L_ON)
  #print(R_ON)


  for i in range(len(phases)):
    if phases[i]==1:
      periods[i][0] -= 1
      periods[i][1] += 1

  #print(phases)
  #print(periods)


  offset = periods[0][0]
  for p in periods:
    p[0]  -= offset
    p[1]  -= offset
  #print(periods)


  poses_corrected = np.zeros_like(poses_np)
  CoMs_corrected = np.zeros_like(CoMs)
  v_p2 = np.array([0,0,0])
  pos_init = np.array([0,0,0])
  contact_xy = []
  for i,p in enumerate(periods):  
    if phases[i]==0:
      #print(poses_np.shape, poses_corrected.shape, CoMs_corrected.shape)
      poses_corrected[p[0]:p[1]+1,...],CoMs_corrected[p[0]:p[1]+1,:],v_p2 = \
        step_Phase1(poses_np[p[0]:p[1]+1,...],CoMs[p[0]:p[1]+1,:],on_foot[i],pos_init)
    else:
      CoM_init = CoMs_corrected[p[0]]
      CoM_end_xy = CoM_init[:2] + v_p2[:2] * (p[1] - p[0]) 
      if on_foot[i-1] == 'L':
        idx_foot = 3
      else: #elif on_foot[i-1] == 'r':
        idx_foot = 6
      # pos_init[2] = 0
      pos_init_xy = CoM_end_xy - CoMs[p[1],:2] + poses_np[p[1],idx_foot,:2]
      pos_init = np.array([pos_init_xy[0],pos_init_xy[1],0])
      contact_xy.append(list(pos_init[:2]))
      #print('init position:',pos_init,'\tv_p2:',v_p2)
  
  #print(CoMs)
  
  result_m1 = np.linalg.norm((contact_xy[-1][0]-contact_xy[-4][0],contact_xy[-1][1]-contact_xy[-4][1]))
  result_m2 = 0
  for i in range(1,4):
    result_m2 += np.linalg.norm((contact_xy[-i][0]-contact_xy[-i-1][0],contact_xy[-i][1]-contact_xy[-i-1][1]))
  print('Predict 1:',result_m1)
  print('Predict 2:',result_m2)
  res1.append(result_m1)
  res2.append(result_m2)
  y.append(info['result'])
  jump.append(info['id'])

In [ ]:
# Save results
np.savez_compressed('Result_Corrected_Pred.npz',id=jump,result=y,pred1=res1,pred2=res2)